In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
package_paths = '../input/pytorch-image-models/pytorch-image-models-master'
import sys
sys.path.append(package_paths)
import os
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import cv2
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import timm
import random
import time
from torch.cuda.amp import autocast, GradScaler

In [ ]:
model_arch = 'tf_efficientnet_b4_ns'
train_rate = 0.8
valid_rate = 0.2
data_root = '../input/cassava-leaf-disease-classification/train_images'
img_size = 512
batch_size = 16
num_workers = 4
seed = 719
epochs = 6
lr = 1e-4
weight_decay = 1e-6
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
CFG = {
    'verbose_step': 1,
    'accum_iter': 2,  # suppoprt to do batch accumulation for backprop with effectively larger batch size
    
    }

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2
train_transforms = Compose([
            RandomResizedCrop(img_size, img_size),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)

valid_transforms = Compose([
            CenterCrop(img_size, img_size, p=1.),
            Resize(img_size, img_size),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def get_image(path):
    img_bgr = cv2.imread(path)
    img_rgb = img_bgr[:,:,::-1]
    return img_rgb

In [ ]:
train_data = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

In [ ]:
imgtype = []

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self,train,data_root,transforms):
        super(CassavaDataset, self).__init__()
        self.train = train
        self.data_root = data_root
        self.transforms = transforms
    
    def __len__(self):
        return self.train.shape[0]
    
    def __getitem__(self, index):
        target = self.train.label[index]
        img = get_image('{}/{}'.format(self.data_root, self.train.loc[index]['image_id']))
        if self.transforms:
            img = self.transforms(image = img)['image']
        imgtype.append(type(img))
        return img, target
        

In [ ]:
class CassavaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super(CassavaImgClassifier, self).__init__()
        self.model = timm.create_model(model_arch,pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
train = train_data[:int(train_data.shape[0]*train_rate)]
valid = train_data.iloc[int(train_data.shape[0]*train_rate):].reset_index(drop=True)

train_DS = CassavaDataset(train,data_root,train_transforms)
valid_DS = CassavaDataset(valid,data_root,valid_transforms)

train_loader = torch.utils.data.DataLoader(train_DS,
                                           batch_size = batch_size,
                                           shuffle = True,
                                           num_workers = num_workers)
valid_loader = torch.utils.data.DataLoader(valid_DS,
                                           batch_size = batch_size,
                                           shuffle = True,
                                           num_workers = num_workers)

In [ ]:
if __name__ == '__main__':
    seed_everything(seed)
    
    model = CassavaImgClassifier(model_arch,
                                     train.label.nunique(), 
                                     pretrained=True).to(device)
    
    scaler = GradScaler()
    optimizer = torch.optim.Adam(model.parameters(),
                                     lr=lr, weight_decay=weight_decay)
        
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1)
        
    loss_fn = nn.CrossEntropyLoss().to(device)
        
    for epoch in range(epochs):
        # 训练
        model.train()
        running_loss = None
        print('Training with {} started'.format(epoch))
        print(len(train), len(valid))
        
        t = time.time()
        pbar = tqdm(enumerate(train_loader), total=len(train_loader))
        for step, (imgs, image_labels) in pbar:
            imgs = imgs.to(device).float()
            image_labels = image_labels.to(device).long()
            
            with autocast():
                image_preds = model(imgs)
                loss = loss_fn(image_preds, image_labels)
                scaler.scale(loss).backward()
                
                if running_loss is None:
                    running_loss = loss.item()
                else:
                    running_loss = running_loss * .99 + loss.item() * .01
                
                if ((step + 1) %  CFG['accum_iter'] == 0) or ((step + 1) == len(train_loader)):
                    # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)

                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad() 
                

                if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(train_loader)):
                    description = f'epoch {epoch} loss: {running_loss:.4f}'
                    pbar.set_description(description)
        
            if scheduler is not None and step%300==0:
                scheduler.step()    
        
        # 验证
        with torch.no_grad():
            model.eval()
            
            t=time.time()
            loss_sum = 0
            sample_num = 0
            image_preds_all = []
            image_targets_all = []
            
            pbar = tqdm(enumerate(valid_loader), total=len(valid_loader))
            for step, (imgs, image_labels) in pbar:
                imgs = imgs.to(device).float()
                image_labels = image_labels.to(device).long()

                image_preds = model(imgs)   #output = model(input)
                #print(image_preds.shape, exam_pred.shape)
                image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
                image_targets_all += [image_labels.detach().cpu().numpy()]

                loss = loss_fn(image_preds, image_labels)

                loss_sum += loss.item()*image_labels.shape[0]
                sample_num += image_labels.shape[0]  

                if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(val_loader)):
                    description = f'epoch {epoch} loss: {loss_sum/sample_num:.4f}'
                    pbar.set_description(description)

            image_preds_all = np.concatenate(image_preds_all)
            image_targets_all = np.concatenate(image_targets_all)
            print('validation multi-class accuracy = {:.4f}'.format((image_preds_all==image_targets_all).mean()))


        torch.save(model.state_dict(),'{}_epoch_{}'.format(model_arch, epoch))

    
    del model, optimizer, train_loader, val_loader, scaler, scheduler
    torch.cuda.empty_cache()
    

In [ ]:
imgtype